In [1]:
import os
import io
import gzip
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import random

from avro.datafile import DataFileReader, DataFileWriter
from avro.io import DatumReader, DatumWriter
import fastavro

from astropy.time import Time
from astropy.io import fits
import os
import glob
from scipy.stats import sigmaclip
import glob

from astropy.coordinates import SkyCoord


In [2]:
#FOR MULTIPLE NIGHTS

# reading avro packages

avro_files= []
night_idx = []
WINTER_detections=[]
m = glob.glob("/data/loki/data/winter/20*")
#for i in range(2):  # uncomment to run on just a few of the nights

for i in range(len(m)): #for all nights!
    idx = m[i].find("20")
    night_id = m[i][idx:]
    path = '/data/loki/data/winter/' + night_id +'/avro/*'
    avro_files.append(glob.glob(path))
    night_idx.append(night_id)

In [3]:
for i in avro_files:
    avro_file = i
    # for each avro package in the folder get the folowing:
    ra,dec,file,packet_idx,winterID = [],[],[],[],[]
    
    
    for avro in avro_file: 
        fname = avro
        with open(fname,'rb') as f: # read the avro package
            freader = fastavro.reader(f) # this has multiple packets
            for i,packet in enumerate(freader):
                
                ra.append(packet['candidate']['ra']) 
                dec.append(packet['candidate']['dec'])
                file.append(fname)
                packet_idx.append(i)
                winterID.append(packet['objectid'])
    
    # saving this in a dataframe
    
    d = {
        'WINTER_name':winterID,
        'ra':ra,
        'dec':dec,
        'file':file,
        'packet_idx':packet_idx
    }
    
    detections = pd.DataFrame(data=d)
    WINTER_detections.append(detections)


In [4]:
all_detections = pd.concat(WINTER_detections)
print(len(all_detections))

good_ones = pd.read_csv("/home/msheshadri/real-bogus/sample_quality/csv/9-9-24-good_cutouts.csv")
samples = []
for i in range(10):
    samples.append(all_detections.iloc[random.randint(0,len(all_detections)-1)])

boguses_idx = []
for k in good_ones['WINTER_name']:
    for idx, WINTER_name in enumerate(samples['WINTER_name']):
        if WINTER_name != k:
            boguses_idx.append(idx)

boguses_idx = list(set(boguses_idx))

bogus_catalog = []
for index in boguses_idx:
    if index < len(all_detections):
        bogus_catalog.append(all_detections.iloc[index])

bogus_catalog= pd.DataFrame(bogus_catalog)
bogus_catalog=bogus_catalog.drop_duplicates()

bogus_catalog.to_csv("./home/msheshadri/real-bogus/bogus/bogus_catalog.csv")

ValueError: No objects to concatenate